We want to isolate households with two parents living under civil union with children being over (or not) 21/25 yo.

In [2]:
secret = False #Will be put to True for a CASD export

In [3]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle

In [148]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/'
Store = pd.HDFStore(store_path)

In [149]:
Store

<class 'pandas.io.pytables.HDFStore'>
File path: C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5

In [150]:
for key in Store.keys(): print(key)
Store.close()

/DECES
/DESCENDANCE
/EAR2004_FAMILLE
/EAR2004_INDIVIDU
/EAR2004_LOGEMENT
/EAR2005_FAMILLE
/EAR2005_INDIVIDU
/EAR2005_LOGEMENT
/EAR2006_FAMILLE
/EAR2006_INDIVIDU
/EAR2006_LOGEMENT
/EAR2007_FAMILLE
/EAR2007_INDIVIDU
/EAR2007_LOGEMENT
/EAR2008_FAMILLE
/EAR2008_INDIVIDU
/EAR2008_LOGEMENT
/EAR2009_FAMILLE
/EAR2009_INDIVIDU
/EAR2009_LOGEMENT
/EAR2010_FAMILLE
/EAR2010_INDIVIDU
/EAR2010_LOGEMENT
/EAR2011_FAMILLE
/EAR2011_INDIVIDU
/EAR2011_LOGEMENT
/EAR2012_FAMILLE
/EAR2012_INDIVIDU
/EAR2012_LOGEMENT
/EAR2013_FAMILLE
/EAR2013_INDIVIDU
/EAR2013_LOGEMENT
/EAR2014_FAMILLE
/EAR2014_INDIVIDU
/EAR2014_LOGEMENT
/EAR2015_FAMILLE
/EAR2015_INDIVIDU
/EAR2015_LOGEMENT
/EAR2016_FAMILLE
/EAR2016_INDIVIDU
/EAR2016_LOGEMENT
/FE
/FISC_INDIVIDU_2011
/FISC_INDIVIDU_2012
/FISC_INDIVIDU_2013
/FISC_INDIVIDU_2014
/FISC_INDIVIDU_2015
/FISC_INDIVIDU_2016
/FISC_LOGEMENT_2011
/FISC_LOGEMENT_2012
/FISC_LOGEMENT_2013
/FISC_LOGEMENT_2014
/FISC_LOGEMENT_2015
/FISC_LOGEMENT_2016
/FISC_REVDET_2011
/FISC_REVDET_2012
/FISC_REVDE

In [7]:
year = 2011

In [8]:
Store.close()
fisc_i = pd.read_hdf( store_path,"FISC_INDIVIDU_2012" )
fisc_revdet = pd.read_hdf( store_path,"FISC_REVDET_2012" )
fisc_revenu = pd.read_hdf(store_path,"FISC_REVENU_2012")
fisc_individu = pd.read_hdf( store_path,"FISC_INDIVIDU_2012" )

In [9]:
fisc_i.shape

(7521834, 32)

In [9]:
print(fisc_i.shape, fisc_revenu.shape, fisc_individu.shape)

(7521834, 32) (2380156, 100) (7521834, 32)


## Data cleaning

All operations needed to process raw data in a good way

In [10]:
fisc_individu["CIDECI"]= fisc_individu.CIDECI.astype("str")

In [11]:
fisc_revenu["ID_FISC_LOG_DIFF"] = fisc_revenu.ID_FISC_LOG_DIFF.astype("float")

In [ ]:
fisc_revenu.sort_values("ID_FISC_LOG_DIFF").head()

In [13]:
(fisc_revenu.NBPERSM - fisc_revenu.NBPAC).value_counts() # 2, soit couples marié, soit célib avec enfant majeur, etc.

2.0     1534517
1.0      504107
3.0      241666
4.0       74490
5.0       16729
6.0        5352
7.0        1779
8.0         776
9.0         347
10.0        191
11.0         98
12.0         49
13.0         26
14.0         12
16.0          6
15.0          6
0.5           3
2.5           1
18.0          1
dtype: int64

### Drop duplicates

On droppe s'il y a plus d'un IDFISC_LOG_DIFF dans fisc_revenu

In [13]:
#Test qu'il n'y a pas de duplicate dans la base
assert (fisc_revenu.ID_FISC_LOG_DIFF.duplicated()).value_counts().index == False

In [14]:
assert(fisc_individu[["ID_FISC_FOY_DIFF", "ORDREFIP", "TYPE_FISC"]
                    ].duplicated()).value_counts().index == False

### Drop not same number of individuals

- Compte le nombre d'individus par logement fiscal dans la table fiscindividu  
- Regarde le nombre de personne dans le logement fiscal (variable NBFOYM)  
- Droppe les logement fiscaux ou le nombre n'est pas exactement le même.

In [15]:
%%time
#compte
nb_pers_by_id_fisclog_in_fisci = fisc_individu.groupby("ID_FISC_LOG_DIFF")["ANAIS"].count();
nb_pers_by_id_fisclog_in_fisci.name = "Nbpers_in_fisci_by_fisclog"
#compare
merge = pd.merge(nb_pers_by_id_fisclog_in_fisci.to_frame().reset_index(),
                 fisc_revenu[["ID_FISC_LOG_DIFF", "NBPERSM", "NBFOYM", "NBPAC"]],
                 on = "ID_FISC_LOG_DIFF" )
print(
    (merge.NBPERSM == merge.Nbpers_in_fisci_by_fisclog).value_counts())
id_fisc_log_to_keep = merge[merge.NBPERSM == merge.Nbpers_in_fisci_by_fisclog].ID_FISC_LOG_DIFF


True     2297044
False      83112
dtype: int64
Wall time: 3.4 s


In [16]:
#fisc_revdet = fisc_revdet[~ fisc_revdet.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_drop)]
fisc_revenu = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_keep)]
fisc_individu = fisc_individu[ fisc_individu.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_keep)]

In [17]:
print(fisc_i.shape, fisc_revenu.shape, fisc_individu.shape)

(7521834, 32) (2297044, 100) (6879111, 32)


## Obtenir la structure familiale

In [256]:
fiscr_2016 = pd.read_hdf(store_path,"FISC_REVENU_2016")
fisci_2016 = pd.read_hdf( store_path,"FISC_INDIVIDU_2016" )

id_log = fisci_2016[((fisci_2016.ANAIS.isin(list(range(1988,2012))))
                    & (fisci_2016.TYPE_FISC.isin(['1','2']))
                    & (fisci_2016.ID_DIFF.notnull()))].ID_FISC_LOG_DIFF
id_diff = fisci_2016[((fisci_2016.ANAIS.isin(list(range(1988,2012))))
                    & (fisci_2016.TYPE_FISC.isin(['1','2']))
                    & (fisci_2016.ID_DIFF.notnull()))].ID_DIFF.values
df = fiscr_2016[fiscr_2016.ID_FISC_LOG_DIFF.isin(id_log)]


234359

In [271]:
df1 = fisci_2016[fisci_2016.ID_DIFF.isin(id_diff)]

In [254]:
df2 = fisc_individu[fisc_individu.ID_DIFF.isin(id_diff)]

In [258]:
(df2.TYPE_FISC.isin(['A','B','C','D', 'E', '3', '4', '5', '6'])).value_counts()

True     77475
False    66883
Name: TYPE_FISC, dtype: int64

In [261]:
id_log_2011 = df2[df2.TYPE_FISC.isin(['A','B','C','D', 'E', '3', '4', '5', '6'])].ID_FISC_LOG_DIFF

In [263]:
fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_log_2011)].ZSALM.head()

0     92119
4     82161
16    49544
28    58680
38    82309
Name: ZSALM, dtype: int64

In [264]:
fiscr_2016[fiscr_2016.ID_FISC_LOG_DIFF.isin(id_log)].ZSALM.head()

13    57869
18    71331
32    17334
33    66623
43    92679
Name: ZSALM, dtype: int64

In [272]:
#Merge fisc fiscr 2011 et 2016
#2011
result_2011 = pd.merge(fisc_revenu,df2[["ID_DIFF", "ID_FISC_LOG_DIFF"]], on = "ID_FISC_LOG_DIFF")
#2016
result_2016 = pd.merge(fiscr_2016,df2[["ID_DIFF", "ID_FISC_LOG_DIFF"]], on = "ID_FISC_LOG_DIFF")

In [283]:
df_final_result = pd.merge(result_2011, result_2016, on = "ID_DIFF", suffixes = ("_2011", "_2016"))

In [296]:
df_temp = df_final_result.query("NBFOYM_2016==1")

In [297]:
reg = sm.OLS.from_formula('ZSALM_2016 ~ ZSALM_2011', data = df_temp )
reg.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ZSALM_2016   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 1.496e+04
Date:                Mon, 09 Jul 2018   Prob (F-statistic):               0.00
Time:                        07:05:02   Log-Likelihood:            -1.8610e+05
No. Observations:               15399   AIC:                         3.722e+05
Df Residuals:                   15397   BIC:                         3.722e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5035.1049    503.007    -10.010      0.000   -6021.059   -4049.151
ZSALM_2011     1.1180      0.009    122.319      0.000       1.100       1.136
==============================================================================
Omnibus:                    52896.884   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      26720812949.666
Skew:                          63.816   Prob(JB):                         0.00
Kurtosis:                    6455.072   Cond. No.                     8.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [288]:
(df_final_result.ID_FISC_LOG_DIFF_2011 == df_final_result.ID_FISC_LOG_DIFF_2016).value_counts()

True     65064
False      224
dtype: int64